## Devices 

> Interfaces to the various smart home providers

In [75]:
# | default_exp devices
# | export
import os

from typing import Any, Optional
from pydantic import BaseModel
from tuya_iot import TuyaOpenAPI

## Base Models

In [76]:
# | export
class Device:
    "Base class for all smart home devices"


class AirPurifier:
    "Air Purifier device"


class Fan(Device):
    "A Fan device"


class CeilingFan(Device):
    "A Fan that is attached to the ceiling"


class TuyaDevice(BaseModel):
    """
    A device controlled by the Tuya Cloud.
    """

    client_id: str
    client_secret: str
    username: str
    password: str
    country_code: str
    endpoint: str = "https://openapi.tuyaus.com"
    appschema: str = "smartLife"
    client: Optional[Any]

    def connect(self):
        self.client = TuyaOpenAPI(self.endpoint, self.client_id, self.client_secret)
        self.client.connect(
            self.username, self.password, self.country_code, self.appschema
        )


class TuyaSceneDevice(TuyaDevice):
    """
    A device controlled via scenes in the Tuya Cloud.
    """

    home_id: str

    def trigger_scene(self, id: str):
        self.client.post(f"/v1.0/homes/{self.home_id}/scenes/{id}/trigger")

    def trigger(self, action: str):
        self.trigger_scene(getattr(self, f"{action}_scene_id"))

In [77]:
from nbdev.showdoc import show_doc

## Air Purifier

We create a `TuyaAirPurifier` that we can use to turn on and off the air purifier.

In [78]:
# | export
class TuyaAirPurifier(TuyaDevice, AirPurifier):
    """Purifiers controlled by a Tuya Smart Switch"""

    device_id: str

    def switch(self, on: bool = True):
        self.client.post(
            f"/v1.0/devices/{self.device_id}/commands",
            {"commands": [{"code": "switch_1", "value": on}]},
        )

    def turn_on(self):
        self.switch(True)

    def turn_off(self):
        self.switch(False)

## Ceiling Fans 

I have a three ceiling fans in the home controlled through RF frequencies.

In [79]:
# | export
class DaikoCeilingFan1(TuyaSceneDevice, CeilingFan):
    light_scene_id: str
    fan_scene_id: str
    fan_1_scene_id: str
    fan_2_scene_id: str
    fan_3_scene_id: str

In [80]:
# | export
living_area_ceiling_fan = DaikoCeilingFan1(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="AniQzKuX7A98n9gn",
    fan_scene_id="tmAFQYbx1oynJcG6",
    fan_1_scene_id="cXSEy5ALHoR4LT5H",
    fan_2_scene_id="fTwRU120YlR0nWOo",
    fan_3_scene_id="hOCfvtwpp1urauns",
)

In [81]:
# | export
class DaikoCeilingFan2(TuyaSceneDevice, CeilingFan):
    light_scene_id: str
    fan_off_scene_id: str
    fan_1_scene_id: str
    fan_2_scene_id: str
    fan_3_scene_id: str

In [83]:
# | export
master_bedroom_ceiling_fan = DaikoCeilingFan2(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="ooGJVKFOQ586ZPDb",
    fan_off_scene_id="tIOVX91BOrpgH86A",
    fan_1_scene_id="JmJpPLVlqhURktzX",
    fan_2_scene_id="Y3tAAqXdEGfarxje",
    fan_3_scene_id="rS34nHG6BRE4ddza",
)

In [84]:
# | export
common_bedroom_ceiling_fan = DaikoCeilingFan2(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="RQB0qD2ec5AqPmT3",
    fan_off_scene_id="RWA1GutcWMWTuaPu",
    fan_1_scene_id="ec4UBLRrVukAm5rG",
    fan_2_scene_id="bHsXR52LReWvhxSL",
    fan_3_scene_id="mg8wiQ4u9WTMy6BM",
)